In [1]:
import torch
import os
import numpy as np
from warnings import simplefilter
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader

simplefilter(action='ignore', category=UserWarning)

In [2]:
MAIN_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(MAIN_DIR, "Solid_droplet", "Data")

In [3]:
from Dataloader import *
from Trainer import *
from Model import *

In [4]:
dtype = torch.float32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# Split the data into train, validation and test set
dataspliter = DataSpliter(DATA_DIR, device, train_frac=0.6, val_frac=0.2, test_frac=0.2, use_seed=True, seed_val=42)

# Load Data from DataLoader
train_image = dataspliter.train
val_image = dataspliter.val
test_image = dataspliter.test
train_labels = dataspliter.train_label
val_labels = dataspliter.val_label
test_labels = dataspliter.test_label

# Create dataset
train_dataset = CustomImageDataset(train_image, train_labels)
val_dataset = CustomImageDataset(val_image, val_labels)
test_dataset = CustomImageDataset(test_image, test_labels)

In [6]:
train_dataloader = DataLoader(train_dataset, batch_size=8)
val_dataloader = DataLoader(val_dataset, batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [7]:
# Get the model
model = Model(device=device, dtype=dtype)
trainer = Trainer(model, train_dataloader, val_dataloader, test_dataloader)

The device that will be used in training is Quadro P1000


In [8]:
trainer.fit(epochs=10, batch_size=8)

Epoch 1


TypeError: can only concatenate list (not "float") to list

In [25]:
loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=False, num_workers=2)

trainer = trainer(model, train_dataloader, val_dataloader, epochs=100)

# dataiter = iter(loader)
# data = dataiter._next_data()
# images, labels = data
# print(images, labels)

# num_epochs = 2
# total_samples = train_dataset.__len__()
# n_iterations = np.ceil(total_samples/1)

# for epoch in range(num_epochs):
#     for i, (images, labels) in enumerate(loader):
#         # forward, backward, update
        


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0') tensor([[41, 32,  1]], dtype=torch.int32)
epoch 1/2, step 5/27.0, inputs torch.Size([1, 320, 256])
epoch 1/2, step 10/27.0, inputs torch.Size([1, 320, 256])
epoch 1/2, step 15/27.0, inputs torch.Size([1, 320, 256])
epoch 1/2, step 20/27.0, inputs torch.Size([1, 320, 256])
epoch 1/2, step 25/27.0, inputs torch.Size([1, 320, 256])
epoch 2/2, step 5/27.0, inputs torch.Size([1, 320, 256])
epoch 2/2, step 10/27.0, inputs torch.Size([1, 320, 256])
epoch 2/2, step 15/27.0, inputs torch.Size([1, 320, 256])
epoch 2/2, step 20/27.0, inputs torch.Size([1, 320, 256])
epoch 2/2, step 25/27.0, inputs torch.Size([1, 320, 256])
